In [108]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import csv

In [187]:
def count_nan_per_column(df):
    nan_count_per_column = df.isna().sum()
    nan_counts = nan_count_per_column[nan_count_per_column > 0]
    if not nan_counts.empty:
        print("Số lượng NaN trên từng cột:")
        print(nan_counts)
    else:
      print("Không có Nan")

In [249]:
def prepare_nan(ds):
  mean_LotFrontage = ds['LotFrontage'].mean()
  ds['LotFrontage'].fillna(mean_LotFrontage, inplace=True)

  ds['GarageYrBlt'].fillna(ds['YearBuilt'], inplace=True)

  fields_to_replace = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
  ds[fields_to_replace] = ds[fields_to_replace].fillna(0)

  return ds

In [178]:
def scalling(ds):
  scaler = MinMaxScaler()
  ds_scaled = scaler.fit_transform(ds)
  ds_scaled = pd.DataFrame(ds_scaled, columns=ds.columns)
  return ds_scaled

In [109]:
import zipfile
import os

def un_zip(zip_file_path):
  extract_to = "/content/"
  folder_name = os.path.splitext(os.path.basename(zip_file_path))[0]

  folder_path = os.path.join(extract_to, folder_name)
  os.makedirs(folder_path, exist_ok=True)

  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)

In [110]:
un_zip('/content/house-prices-advanced-regression-techniques.zip')

In [238]:
train_ds = pd.read_csv('/content/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
test_ds = pd.read_csv('/content/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

In [242]:
train_columns_numeric = train_ds.select_dtypes(include=['int64', 'float64']).columns
train_ds = train_ds[train_columns_numeric]

test_columns_numeric = test_ds.select_dtypes(include=['int64', 'float64']).columns
test_ds = test_ds[test_columns_numeric]

In [244]:
count_nan_per_column(train_ds)
count_nan_per_column(test_ds)

Số lượng NaN trên từng cột:
LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64
Số lượng NaN trên từng cột:
LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64


In [250]:
train_ds = prepare_nan(train_ds)
test_ds = prepare_nan(test_ds)

<ipython-input-249-48e93baf91d7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['LotFrontage'].fillna(mean_LotFrontage, inplace=True)
<ipython-input-249-48e93baf91d7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['GarageYrBlt'].fillna(ds['YearBuilt'], inplace=True)
<ipython-input-249-48e93baf91d7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds[fields_to_replace] =

In [251]:
train_ds

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,40,0,0,0,0,0,8,2007,175000
1457,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,349,0,0,0,0,0,0,2,2010,210000
1458,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,60,0,0,0,0,2500,5,2010,266500


In [252]:
X_train = train_ds.drop('SalePrice', axis=1)
y_train = train_ds['SalePrice']

X_test = test_ds

In [253]:
X_train = scalling(X_train)
X_test = scalling(X_test)

In [183]:
y_train

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1371, dtype: int64

In [255]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.122500,0.125089,0.000000,...,0.386460,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.00000,0.090909,0.50
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.000000,0.173281,0.000000,...,0.324401,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.363636,0.25
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.101250,0.086109,0.000000,...,0.428773,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.00000,0.727273,0.50
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.000000,0.038271,0.000000,...,0.452750,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.00000,0.090909,0.00
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.218750,0.116052,0.000000,...,0.589563,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.00000,1.000000,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.140411,0.030929,0.555556,0.500,0.920290,0.833333,0.000000,0.000000,0.000000,...,0.324401,0.000000,0.073126,0.000000,0.0,0.0,0.0,0.00000,0.636364,0.25
1456,0.000000,0.219178,0.055505,0.555556,0.625,0.768116,0.633333,0.074375,0.139972,0.110583,...,0.352609,0.407235,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.090909,1.00
1457,0.294118,0.154110,0.036187,0.666667,1.000,0.500000,0.933333,0.000000,0.048724,0.000000,...,0.177715,0.000000,0.109689,0.000000,0.0,0.0,0.0,0.16129,0.363636,1.00
1458,0.000000,0.160959,0.039342,0.444444,0.625,0.565217,0.766667,0.000000,0.008682,0.698100,...,0.169252,0.427071,0.000000,0.202899,0.0,0.0,0.0,0.00000,0.272727,1.00


In [281]:
y_train

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64

In [254]:
X_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.000000,0.329609,0.184147,0.444444,0.625,0.625954,0.183333,0.000000,0.116708,0.094364,...,0.490591,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0
1,0.000000,0.335196,0.232124,0.555556,0.625,0.603053,0.133333,0.083721,0.230175,0.000000,...,0.209677,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0
2,0.235294,0.296089,0.224197,0.444444,0.500,0.900763,0.800000,0.000000,0.197257,0.000000,...,0.323925,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0
3,0.235294,0.318436,0.154326,0.555556,0.625,0.908397,0.800000,0.015504,0.150125,0.000000,...,0.315860,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0
4,0.588235,0.122905,0.064121,0.777778,0.500,0.862595,0.700000,0.000000,0.065586,0.000000,...,0.340054,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.823529,0.000000,0.008453,0.333333,0.750,0.694656,0.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.454545,0.0
1455,0.823529,0.000000,0.007691,0.333333,0.500,0.694656,0.333333,0.000000,0.062843,0.000000,...,0.192204,0.000000,0.032345,0.0,0.0,0.000000,0.0,0.000000,0.272727,0.0
1456,0.000000,0.776536,0.336115,0.444444,0.750,0.618321,0.766667,0.000000,0.305237,0.000000,...,0.387097,0.332865,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.727273,0.0
1457,0.382353,0.229050,0.162724,0.444444,0.500,0.862595,0.700000,0.000000,0.084040,0.000000,...,0.000000,0.056180,0.043127,0.0,0.0,0.000000,0.0,0.041176,0.545455,0.0


In [277]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import backend as K
import tensorflow as tf

def rmse(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential(
    [
        Dense(4, input_shape=[36], activation='linear'),
        Dense(1, activation='linear')
    ]
)

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=1500)

Epoch 1/1500
46/46 [==============================] - 1s 2ms/step - loss: 39039078400.0000
Epoch 2/1500
46/46 [==============================] - 0s 2ms/step - loss: 39038734336.0000
Epoch 3/1500
46/46 [==============================] - 0s 2ms/step - loss: 39038279680.0000
Epoch 4/1500
46/46 [==============================] - 0s 2ms/step - loss: 39037677568.0000
Epoch 5/1500
46/46 [==============================] - 0s 2ms/step - loss: 39036907520.0000
Epoch 6/1500
46/46 [==============================] - 0s 2ms/step - loss: 39035990016.0000
Epoch 7/1500
46/46 [==============================] - 0s 2ms/step - loss: 39034892288.0000
Epoch 8/1500
46/46 [==============================] - 0s 2ms/step - loss: 39033634816.0000
Epoch 9/1500
46/46 [==============================] - 0s 2ms/step - loss: 39032209408.0000
Epoch 10/1500
46/46 [==============================] - 0s 2ms/step - loss: 39030636544.0000
Epoch 11/1500
46/46 [==============================] - 0s 2ms/step - loss: 39028887552.00

In [278]:
y_pred = model.predict(X_test)

46/46 [==============================] - 0s 1ms/step


In [279]:
print(y_pred)
print(y_pred.shape)

[[145139.7 ]
 [169431.02]
 [191395.69]
 ...
 [182851.14]
 [143669.81]
 [205149.4 ]]
(1459, 1)


In [ ]:
id = [i for i in range(1461, 1461+y_pred.shape[0])]

file_name = "submition.csv"
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Id', 'SalePrice'])

    for i in range(len(id)):
        writer.writerow([id[i], y_pred_sk[i][0]])

In [282]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

LinearRegression()

In [286]:
y_pred_sk = model.predict(X_test)

In [287]:
id = [i for i in range(1461, 1461+y_pred.shape[0])]

file_name = "submition_linReg.csv"
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Id', 'SalePrice'])

    for i in range(len(id)):
        writer.writerow([id[i], y_pred_sk[i]])